In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from mpl_toolkits.mplot3d import Axes3D

# Dupire's Formula
#### $\sigma(T,K)^2 = \frac{2}{K^2}{\frac{\frac{\partial c(T,K)}{\partial T} + rK\frac{\partial c}{\partial K}}{\frac{\partial^2 c}{\partial K^2}}}$
Setting $r = 0$ we get:
#### $\sigma(T,K)^2 = \frac{2}{K^2}{\frac{\frac{\partial c(T,K)}{\partial T}}{\frac{\partial^2 c}{\partial K^2}}}$

# Load in options data

In [2]:
options_1 = pd.read_csv('./New_option_prices_1.csv', header=None, names=["Maturity", "Strike", "Price"])
options_2 = pd.read_csv('./New_option_prices_2.csv', header=None, names=["Maturity", "Strike", "Price"])

options_1 = options_2
options_1['Price'] = options_1['Price'] + np.random.normal(0,0.001,105)

## Approximate using finite differences
Using finite diffrences we can approximate the derivatives as follows:
#### Forward Difference
#### $\frac {\partial c(T,K)}{\partial T} \approx \frac{c(T + h, K) - c(T, K)}{h}$ and $\frac {\partial c(T,K)}{\partial K} \approx \frac{c(T, K + h) - c(T,K)}{h}$

#### Backward Difference
#### $\frac {\partial c(T,K)}{\partial T} \approx \frac{c(T , K) - c(T - h, K)}{h}$ and $\frac {\partial c(T,K)}{\partial K} \approx \frac{c(T, K ) - c(T,K - h)}{h}$

#### Central Difference
#### $\frac {\partial c(T,K)}{\partial T} \approx \frac{c(T + h_1, K) - c(T - h_2, K)}{h_1 + h_2}$ and $\frac {\partial c(T,K)}{\partial K} \approx \frac{c(T, K + h_1 ) - c(T,K - h_2)}{h_1 + h_2}$

In [3]:
#options_1['dt_backwards'] = options_1.groupby('Strike').apply(lambda x: x['Price'].diff(-1)/x['Maturity'].diff(-1)).reset_index(level=0)[0]
options_1['dt_forwards'] = options_1.groupby('Strike').apply(lambda x: x['Price'].diff(-1)/x['Maturity'].diff(-1)).reset_index(level=0)[0]
options_1['dK2'] = options_1[['Maturity', 'Strike','Price']].groupby('Maturity').apply(lambda x: (x['Price'].shift(1) - 2*x['Price'] + x['Price'].shift(-1))/100).reset_index(level=0)['Price']
options_1['iv'] = np.sqrt(2*options_1['dt_forwards']/(options_1['Strike']**2 *options_1['dK2']))

In [8]:
#%matplotlib widget

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# Make data.
X = options_1['Maturity'].unique()
Y = options_1['Strike'].unique()
X, Y = np.meshgrid(X, Y)
Z = []
for i in range(5):
    Z.append(options_1['iv'].iloc[i*21:(i+1)*21].values)
Z = np.transpose(np.array(Z))

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, linewidth=0, antialiased=False)

# Customize the z axis.
ax.zaxis.set_major_locator(LinearLocator(10))
# A StrMethodFormatter is used automatically

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\Tariq\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:15: UserWarning: Z contains NaN values. This may result in rendering artifacts.
  from ipykernel import kernelapp as app


In [ ]:
plt.plot(options_1['Maturity'].unique(), Z[5])

#### $C = N(d_1)S_t - N(d_2)Ke^{-rt}$
#### $d_1 = \frac{ln \frac{S_t}{K} + (r + \frac{\sigma^2}{2})(T-t)}{\sigma \sqrt{T-t}}$
#### $d_2 = d_1 - \sigma \sqrt{T-t}$

#### $C$ = Call options price
#### $N$ = CDF Of the normal districbution
#### $S_t$ = Spot price of and asset
#### $K$ = Strike price
#### $r$ = risk-free interest rate
#### $t$ = time to maturity
#### $\sigma$ = volatility of asset